<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Physical-model" data-toc-modified-id="Physical-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Physical model</a></span></li><li><span><a href="#Optimisation-probleme-formulation" data-toc-modified-id="Optimisation-probleme-formulation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Optimisation probleme formulation</a></span></li><li><span><a href="#Solve-Problem" data-toc-modified-id="Solve-Problem-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Solve Problem</a></span></li><li><span><a href="#Brut-Force-approach" data-toc-modified-id="Brut-Force-approach-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Brut Force approach</a></span></li></ul></div>

In [ ]:
# %matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import ipywidgets as ipw
from matplotlib import cm
import ipywidgets as widgets
from scipy import integrate
import pandas as pd

%matplotlib widget

## Physical model

In [ ]:
A = np.pi * 0.05**2  #  cross area [m**2]
cx = 0.62  # Drag coefficient []
m = 2  # mass [kg]
rho = 1.225  # Air density [kg/m**3]
g = 9.81  # Gravity [m/s**2]
windx, windy = -30.0, 00.0
fly_duration = 5.0  # seconde

# target
target = np.array([80, 50])


t = np.linspace(0.0, fly_duration, 200)


def derivative(X, t):
    """
    Target ODE: Newton's second law
    """
    x, y, vx, vy = X
    # relativ speed for aero
    vxr = vx - windx
    vyr = vy - windy
    v = (vxr**2 + vyr**2) ** 0.5
    Tx, Ty = vxr / v, vyr / v
    ax = -0.5 * rho * v**2 * A * cx * Tx / m
    ay = -0.5 * rho * v**2 * A * cx * Ty / m - g
    return np.array([vx, vy, ax, ay])


def eval_sol(theta0, v0, target):
    x0, y0 = 0.0, 0.0
    X0 = [x0, y0, v0 * np.cos(np.radians(theta0)), v0 * np.sin(np.radians(theta0))]
    sol = integrate.odeint(derivative, X0, t)
    out = pd.DataFrame(sol, columns=["x", "y", "vx", "vy"])

    dist = np.linalg.norm(target - sol[:, 0:2], axis=1)
    dist = np.min(dist)

    dist = np.sqrt(((target - sol[-1, 0:2]) ** 2).sum())

    return out, dist

In [ ]:
theta0 = 45.0
v0 = 10.0  # Initial velocity [m/s]
out, dist = eval_sol(theta0, v0, target)

In [ ]:
plt.figure()
(l,) = plt.plot(out.x, out.y)
(lend,) = plt.plot(out.x.values[-1], out.y.values[-1], "*y")
plt.plot(target[0], target[1], "or")
plt.grid()
plt.ylim(0.0, target[1] * 2)
plt.xlim(0.0, target[0] * 2)
plt.xlabel("Position, $x$")
plt.ylabel("Position, $y$")
plt.show()


@ipw.interact(
    v0=(0.0, 100, 1), theta0=(0, 90, 2.0), algorithm=["Nelder-Mead", "BFGS", "Powell"]
)
def update_plot(theta0, v0):
    out, dist = eval_sol(theta0, v0, target)
    l.set_data(out.x, out.y)
    lend.set_data(out.x.values[-1], out.y.values[-1])

In [ ]:
theta0 = 45.0
v0 = 10.0  # Initial velocity [m/s]
out, dist = eval_sol(theta0, v0, target)
dist

In [ ]:
plt.figure()
plt.plot(out.x, out.y)
plt.plot(out.x.values[-1], out.y.values[-1], "*y")
plt.plot(target[0], target[1], "or")
plt.grid()
plt.ylim(0.0, 50.0)
plt.xlabel("Position, $x$")
plt.ylabel("Position, $y$")
plt.show()

## Optimisation probleme formulation

In [ ]:
def cost(Param):
    v0 = Param[1]
    theta0 = Param[0]
    out, dist = eval_sol(theta0, v0, target)
    return dist

## Solve Problem

In [ ]:
X0 = [45, 10]
sol = optimize.minimize(cost, X0, method="BFGS")
sol.x

In [ ]:
theta0 = sol.x[0]
v0 = sol.x[1]
out, dist = eval_sol(theta0, v0, target)
dist

In [ ]:
plt.figure()
plt.plot(target[0], target[1], "or")
plt.plot(out.x, out.y)
plt.plot(out.x.values[-1], out.y.values[-1], "*y")

plt.grid()
plt.ylim(0.0, 500.0)
plt.xlabel("Position, $x$")
plt.ylabel("Position, $y$")
plt.show()

## Brut Force approach

In [ ]:
Ntheta, Nvit = 45, 200
theta = np.linspace(0.0, 90, Ntheta)
v0 = np.linspace(10, 300, Nvit)
X, Y = np.meshgrid(theta, v0)
param = np.array([X.flatten(), Y.flatten()])
zf = np.zeros_like(X.flatten())

for i in range(len(param.T)):
    zf[i] = cost(param[:, i])

Z = zf.reshape(Nvit, Ntheta)
# Z = np.where(Z > 200.0, np.nan, Z)

In [ ]:
plt.figure(figsize=(8, 6))
title = plt.title("")
cost_map = plt.contourf(X, Y, Z, 50, cmap=cm.jet)
plt.colorbar(cost_map)
plt.contour(X, Y, Z, 50, cmap=cm.gray, zorder=2.0)
plt.xlabel(r"$\theta$")
plt.ylabel("V0")

In [ ]:
from scipy import interpolate

In [ ]:
nx, ny = 50, 20
world = np.ones((ny, nx))
world[:, :20] = 2
world[:, 30:] = 2

In [ ]:
X = np.array([[0, 0], [20, 0], [25, 12], [30, 10], [50, 20]])

dist = np.linalg.norm(np.diff(X, axis=0), axis=1)
center = X[:-1] + np.diff(X, axis=0) / 2

np.inter2d

Speed = np.array([5, 4, 1, 5])

t = dist / Speed
t_total = t.sum()

plt.figure()
plt.plot(X[:, 0], X[:, 1], ":o")
plt.plot(center[:, 0], center[:, 1], "xr")
plt.imshow(world)

In [ ]:
plt.figure()
(l,) = plt.plot(X[:, 0], X[:, 1], ":o")
(lc,) = plt.plot(center[:, 0], center[:, 1], "xr")
plt.imshow(world)


@ipw.interact(y1=(0.0, 20, 1), y2=(0.0, 20, 1))
def update(y1, y2):
    X = np.array([[0, 0], [20, y1], [25, y2], [30, 10], [50, 20]])
    dist = np.linalg.norm(np.diff(X, axis=0), axis=1)
    center = X[:-1] + np.diff(X, axis=0) / 2
    Speed = np.array([5, 3, 1, 5])
    t = dist / Speed
    t_total = t.sum()
    l.set_data(X[:, 0], X[:, 1])
    lc.set_data(center[:, 0], center[:, 1])